# 0.0 IMPORT

In [1]:
import pandas as pd

from sqlalchemy import create_engine

## 0.1 Coleta de dados

In [2]:
path = '/home/borges/'
database_name = 'database_hm.sqlite'
conn = create_engine('sqlite:///' + path + database_name, echo=False)

In [3]:
query = """
    SELECT * FROM vitrine
"""

In [4]:
df_raw = pd.read_sql(query, con=conn)

In [5]:
df_raw.sample(8)

,product_id,style_id,color_id,product_name,color_name,fit,product_pric,size_number,size_model,cotton,polyester,elastane,elasterell,scrapy_datetime
428,0427159005,0427159,005,trashed_skinny_jeans,dark_denim_blue,skinny_fit,39.99,None,None,0.72,0.20,0.01,0.00,2021-09-07 10:14:25
137,0427159007,0427159,007,trashed_skinny_jeans,black_washed_out,skinny_fit,18.99,None,None,0.91,0.07,0.02,0.00,2021-09-06 19:25:43
203,1008549004,1008549,004,regular_jeans,dark_blue,regular_fit,19.99,189,32/32,0.99,0.63,0.01,0.00,2021-09-06 19:41:22
1117,0811993036,0811993,036,regular_jeans,blue,regular_fit,29.99,183,31/32,0.99,0.00,0.01,0.00,2021-09-08 09:52:51
402,0811993024,0811993,024,regular_jeans,denim_blue,regular_fit,29.99,None,None,1.00,0.00,0.01,0.00,2021-09-07 10:14:25
900,0814631003,0814631,003,freefit®_slim_jeans,dark_gray,slim_fit,27.99,None,None,0.90,0.00,0.02,0.08,2021-09-07 14:59:00
12,0720504001,0720504,001,skinny_jeans,black,skinny_fit,14.99,None,None,0.80,1.00,0.01,0.00,2021-09-06 19:25:43
1027,0636207006,0636207,006,slim_jeans,black,slim_fit,19.99,None,None,0.98,0.00,0.02,0.00,2021-09-08 09:52:51


# 1.0 Passo 01 - Descrição dos dados

In [6]:
df01 = df_raw.copy()

## 1.1 Dimesão dos dados

In [7]:
print('Number of Rows: {}'.format(df01.shape[0]))
print('Number of Columns: {}'.format(df01.shape[1]))

Number of Rows: 1168
Number of Columns: 14


## 1.2 Tipos de dados

In [8]:
df01.dtypes

product_id          object
style_id            object
color_id            object
product_name        object
color_name          object
fit                 object
product_pric       float64
size_number         object
size_model          object
cotton             float64
polyester          float64
elastane           float64
elasterell         float64
scrapy_datetime     object
dtype: object

In [9]:
# convert object to datetime
df01['scrapy_datetime'] = pd.to_datetime(df01['scrapy_datetime'])

## 1.3 Idetificação de dados faltantes

In [10]:
df01.isna().sum()

product_id           0
style_id             0
color_id             0
product_name         0
color_name           0
fit                  0
product_pric         0
size_number        918
size_model         934
cotton               0
polyester            0
elastane             0
elasterell           0
scrapy_datetime      0
dtype: int64

In [12]:
df01.isna().sum() / df01.shape[0]

product_id         0.000000
style_id           0.000000
color_id           0.000000
product_name       0.000000
color_name         0.000000
fit                0.000000
product_pric       0.000000
size_number        0.785959
size_model         0.799658
cotton             0.000000
polyester          0.000000
elastane           0.000000
elasterell         0.000000
scrapy_datetime    0.000000
dtype: float64

## 1.4 Substituição dos dados faltantes

## 1.5 Descrição dos dados